**Task 07: Querying RDF(s)**

In [2]:
!pip install rdflib
import urllib.request
url = 'https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2025-2026/refs/heads/master/Assignment4/course_materials/python/validation.py'
urllib.request.urlretrieve(url, 'validation.py')
github_storage = "https://raw.githubusercontent.com/FacultadInformatica-LinkedData/Curso2025-2026/master/Assignment4/course_materials"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.4/565.4 kB 10.1 MB/s eta 0:00:00


In [3]:
from validation import Report

First let's read the RDF file

In [4]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS
# Do not change the name of the variables
g = Graph()
g.namespace_manager.bind('ns', Namespace("http://somewhere#"), override=False)
g.parse(github_storage+"/rdf/data06.ttl", format="TTL")
report = Report()

**TASK 7.1a: For all classes, list each classURI. If the class belogs to another class, then list its superclass.**
**Do the exercise in RDFLib returning a list of Tuples: (class, superclass) called "result". If a class does not have a super class, then return None as the superclass**

In [5]:
# TO DO

def listar_clases_y_superclase(grafo):
    clases = set(grafo.subjects(RDF.type, RDFS.Class))
    clases.update(grafo.subjects(RDFS.subClassOf, None))   # subclases
    clases.update(grafo.objects(None, RDFS.subClassOf))    # superclases

    resultado = []
    for c in clases:
        sc = grafo.value(subject=c, predicate=RDFS.subClassOf, object=None)
        resultado.append((c, sc))
    return resultado

# Visualize the results
result = listar_clases_y_superclase(g) #list of tuples
for r in result:
  print(r)

(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Animal'), None)
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#FullProfessor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Professor'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Student'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Person'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Professor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Person'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#AssociateProfessor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Professor'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#InterimAssociateProfessor'), rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#AssociateProfessor'))
(rdflib.term.URIRef('http://oeg.fi.upm.es/def/people#Person'), None)


In [6]:
## Validation: Do not remove
report.validate_07_1a(result)

TASK 7.1a OK


**TASK 7.1b: Repeat the same exercise in SPARQL, returning the variables ?c (class) and ?sc (superclass)**

In [9]:
query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?c ?sc
WHERE {
  {
    { ?c a rdfs:Class }
    UNION
    { ?c rdfs:subClassOf ?_any }
    UNION
    { ?_x rdfs:subClassOf ?c }
  }
  OPTIONAL { ?c rdfs:subClassOf ?sc }  # su superclase si existe
}
"""

for r in g.query(query):
  print(r.c, r.sc)


http://oeg.fi.upm.es/def/people#Person None
http://oeg.fi.upm.es/def/people#Animal None
http://oeg.fi.upm.es/def/people#Professor http://oeg.fi.upm.es/def/people#Person
http://oeg.fi.upm.es/def/people#Student http://oeg.fi.upm.es/def/people#Person
http://oeg.fi.upm.es/def/people#FullProfessor http://oeg.fi.upm.es/def/people#Professor
http://oeg.fi.upm.es/def/people#AssociateProfessor http://oeg.fi.upm.es/def/people#Professor
http://oeg.fi.upm.es/def/people#InterimAssociateProfessor http://oeg.fi.upm.es/def/people#AssociateProfessor


In [10]:
## Validation: Do not remove
report.validate_07_1b(query,g)

TASK 7.1b OK


**TASK 7.2a: List all individuals of "Person" with RDFLib (remember the subClasses). Return the individual URIs in a list called "individuals"**


In [11]:
ns = Namespace("http://oeg.fi.upm.es/def/people#")

person_class = ns.Person

def get_all_subclasses(cls):
    subclasses = set(g.subjects(RDFS.subClassOf, cls))
    for sc in list(subclasses):
        subclasses |= get_all_subclasses(sc)
    return subclasses

all_classes = {person_class} | get_all_subclasses(person_class)

# variable to return
individuals = []
for c in all_classes:
    for ind in g.subjects(RDF.type, c):
        individuals.append(ind)
# visualize results
for i in individuals:
  print(i)

http://oeg.fi.upm.es/def/people#Oscar
http://oeg.fi.upm.es/def/people#Asun
http://oeg.fi.upm.es/def/people#Raul


In [12]:
# validation. Do not remove
report.validate_07_02a(individuals)

TASK 7.2a OK


**TASK 7.2b: Repeat the same exercise in SPARQL, returning the individual URIs in a variable ?ind**

In [13]:
query = """
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ns:   <http://oeg.fi.upm.es/def/people#>

SELECT DISTINCT ?ind
WHERE {
  ?c rdfs:subClassOf* ns:Person .
  ?ind rdf:type ?c .
}
"""

for r in g.query(query):
  print(r.ind)
# Visualize the results

http://oeg.fi.upm.es/def/people#Asun
http://oeg.fi.upm.es/def/people#Oscar
http://oeg.fi.upm.es/def/people#Raul


In [14]:
## Validation: Do not remove
report.validate_07_02b(g, query)

TASK 7.2b OK


**TASK 7.3:  List the name and type of those who know Rocky (in SPARQL only). Use name and type as variables in the query**

In [18]:
query = """
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ns:   <http://oeg.fi.upm.es/def/people#>

SELECT DISTINCT ?name ?type
WHERE {
  ?ind ?p ns:Rocky .
  ?ind rdfs:label ?name .
  ?ind rdf:type ?type .
}
"""
# Visualize the results
for r in g.query(query):
  print(r.name, r.type)


Asun http://oeg.fi.upm.es/def/people#FullProfessor
Raul http://oeg.fi.upm.es/def/people#InterimAssociateProfessor
Fantasma http://oeg.fi.upm.es/def/people#Animal


In [19]:
## Validation: Do not remove
report.validate_07_03(g, query)

TASK 7.3 OK


**Task 7.4: List the name of those entities who have a colleague with a dog, or that have a collegue who has a colleague who has a dog (in SPARQL). Return the results in a variable called name**

In [32]:
query = """
PREFIX rdf:  <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX ns:   <http://oeg.fi.upm.es/def/people#>

SELECT DISTINCT ?name
WHERE {
  ?ind rdfs:label ?name .

  {
    ?ind ns:hasColleague ?c .
  }
  UNION
  {
    ?ind ns:hasColleague/ns:hasColleague ?c .
  }

  ?c ?any ns:Rocky .
}
"""

for r in g.query(query):
  print(r.name)

# TO DO
# Visualize the results

Asun
Oscar
Raul


In [33]:
## Validation: Do not remove
report.validate_07_04(g,query)
report.save_report("_Task_07")

TASK 7.4 OK
